# Import related pacakages

In [5]:
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

In [2]:
import pandas as pd
ECMpy_solution_frame=pd.read_csv('./analysis/ECMpy_ori_solution_df_pfba.csv',index_col=0)
ECMpy_adj_solution_frame=pd.read_csv('./analysis/ECMpy_adj_round1_solution_df_pfba.csv',index_col=0)
Orimodel_solution_frame=pd.read_csv('./data/C13reaction.csv' ,index_col=0)
insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_fluxes_round1.svg"
draw_calibration_kcat_figure(Orimodel_solution_frame,ECMpy_solution_frame,ECMpy_adj_solution_frame,insvg,outsvg)

In [3]:
import pandas as pd
ECMpy_solution_frame=pd.read_csv('./analysis/ECMpy_ori_solution_df_pfba.csv',index_col=0)
ECMpy_adj_solution_frame=pd.read_csv('./analysis/ECMpy_adj_round2_solution_df_pfba.csv',index_col=0)
Orimodel_solution_frame=pd.read_csv('./data/C13reaction.csv' ,index_col=0)
insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_fluxes_round2.svg"
draw_calibration_kcat_figure(Orimodel_solution_frame,ECMpy_solution_frame,ECMpy_adj_solution_frame,insvg,outsvg)

In [6]:
import pandas as pd
ECMpy_solution_frame=pd.read_csv('./analysis/ECMpy_ori_solution_df_pfba.csv',index_col=0)
ECMpy_adj_solution_frame=pd.read_csv('./analysis/ECMpy_autopacmen_adj_manual_solution_df_pfba.csv',index_col=0)
Orimodel_solution_frame=pd.read_csv('./data/C13reaction.csv' ,index_col=0)
insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_fluxes_manual.svg"
draw_calibration_kcat_figure(Orimodel_solution_frame,ECMpy_solution_frame,ECMpy_adj_solution_frame,insvg,outsvg)

In [ ]:
与GECKO,smoment比较

In [2]:
import cobra
import pandas as pd
model_gecko_adj_subunit=cobra.io.read_sbml_model( "./model/iML1515_new_GECKO_20201129_subunit.xml")
model_smoment_adj_subunit=cobra.io.read_sbml_model( "./model/iML1515_new_sMOMENT_20201129_subunit.xml")

model_gecko_adj_subunit.reactions.get_by_id("ER_pool_TG_").upper_bound=0.228#0.095
model_smoment_adj_subunit.reactions.get_by_id("ER_pool_TG_").upper_bound=0.228#0.095
model_gecko_adj_subunit_solution = cobra.flux_analysis.pfba(model_gecko_adj_subunit)
model_gecko_adj_subunit_solution_frame = model_gecko_adj_subunit_solution.to_frame()
model_gecko_adj_subunit_solution_frame.to_csv('./analysis/model_gecko_adj_subunit_solution_pfba.csv')
model_smoment_adj_subunit_solution = cobra.flux_analysis.pfba(model_smoment_adj_subunit)
model_smoment_adj_subunit_solution_frame = model_smoment_adj_subunit_solution.to_frame()
model_smoment_adj_subunit_solution_frame.to_csv('./analysis/model_smoment_adj_subunit_solution_pfba.csv')

Orimodel_solution_frame=pd.read_csv('./analysis/Orimodel_solution_df_pfba.csv',index_col=0)
ECMpy_solution_frame=pd.read_csv('./analysis/ECMpy_autopacmen_adj_manual_solution_df_pfba.csv',index_col=0)
C13_solution_frame=pd.read_csv('./data/C13reaction.csv' ,index_col=0)

import re
import numpy as np
total_solution_frame=pd.DataFrame()

for reaction in Orimodel_solution_frame.index:
    if re.search('_num',reaction):
        eachreaction = reaction.split('_num')[0]
    else:
        eachreaction = reaction
    total_solution_frame.loc[eachreaction,'reaction']=eachreaction
    if eachreaction in C13_solution_frame.index:
        total_solution_frame.loc[eachreaction,'C13']=C13_solution_frame.loc[eachreaction,'Flux norm']
    else:
        pass
    total_solution_frame.loc[eachreaction,'Orimodel_fluxes']=Orimodel_solution_frame.loc[reaction,'fluxes']

    ECMpy_list=[]
    for reaction in ECMpy_solution_frame.index:
        if re.search('_num',reaction):
            search_reaction = reaction.split('_num')[0]
        else:
            search_reaction = reaction
        if eachreaction==search_reaction:
            ECMpy_list.append(ECMpy_solution_frame.loc[reaction,'fluxes'])
    if len(ECMpy_list)>0:
        total_solution_frame.loc[eachreaction,'ECMpy_fluxes']=np.max(ECMpy_list)

    gecko_list=[]
    for reaction in model_gecko_adj_subunit_solution_frame.index:
        search_reaction=eachreaction+'_'
        if eachreaction==reaction:
            gecko_list.append(model_gecko_adj_subunit_solution_frame.loc[reaction,'fluxes'])
        elif re.search(search_reaction,reaction):
            gecko_list.append(model_gecko_adj_subunit_solution_frame.loc[reaction,'fluxes'])   
    if len(gecko_list)>0:
        total_solution_frame.loc[eachreaction,'model_gecko_adj_subunit_fluxes']=np.max(gecko_list)

    smoment_list=[]
    for reaction in model_smoment_adj_subunit_solution_frame.index:
        search_reaction=eachreaction+'_'
        if eachreaction==reaction:
            smoment_list.append(model_smoment_adj_subunit_solution_frame.loc[reaction,'fluxes'])
        elif re.search(search_reaction,reaction):
            smoment_list.append(model_smoment_adj_subunit_solution_frame.loc[reaction,'fluxes'])
    if len(smoment_list)>0:        
        total_solution_frame.loc[eachreaction,'model_smoment_adj_subunit_fluxes']=np.max(smoment_list)    
    
total_solution_frame.to_csv('./analysis/iML1515_different_model_cb.csv')

In [6]:
infile='./analysis/iML1515_different_model_cb.csv'
model_data=pd.read_csv(infile,index_col=0)
insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_fluxes_diff_model.svg"
draw_different_model_cb_figure2(model_data,insvg,outsvg)

In [6]:
json_model_path="./model/iML1515_irr_enz_constraint_adj_round2.json"
enz_model=get_enzyme_constraint_model(json_model_path)
iML1515=cobra.io.read_sbml_model( "./model/iML1515_new.xml")
model_gecko_adj_subunit=cobra.io.read_sbml_model( "./model/iML1515_new_GECKO_20201129_subunit.xml")
model_smoment_adj_subunit=cobra.io.read_sbml_model( "./model/iML1515_new_sMOMENT_20201129_subunit.xml")

print('ECMpy reactions : '+str(len(enz_model.reactions)))
print('GECKO reactions : '+str(len(model_gecko_adj_subunit.reactions)))
print('sMOMENT reactions : '+str(len(model_smoment_adj_subunit.reactions)))
print('iML1515 reactions : '+str(len(iML1515.reactions)))

print('ECMpy metabolites : '+str(len(enz_model.metabolites)))
print('GECKO metabolites : '+str(len(model_gecko_adj_subunit.metabolites)))
print('sMOMENT metabolites : '+str(len(model_smoment_adj_subunit.metabolites)))
print('iML1515 metabolites : '+str(len(iML1515.metabolites)))

print('ECMpy genes : '+str(len(enz_model.genes)))
print('GECKO genes : '+str(len(model_gecko_adj_subunit.genes)))
print('sMOMENT genes : '+str(len(model_smoment_adj_subunit.genes)))
print('iML1515 genes : '+str(len(iML1515.genes)))

ECMpy reactions : 5883
GECKO reactions : 8447
sMOMENT reactions : 3336
iML1515 reactions : 2713
ECMpy metabolites : 1877
GECKO metabolites : 4494
sMOMENT metabolites : 1878
iML1515 metabolites : 1877
ECMpy genes : 1516
GECKO genes : 1516
sMOMENT genes : 1516
iML1515 genes : 1516
